In [ ]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer 
import random
import io
import string
from sklearn.metrics.pairwise import cosine_similarity
import MeCab 
import numpy as np

In [ ]:
mecab = MeCab.Tagger('-Owakati')

#p = Path("/home/ifte/amiebot_project/amie-HelpBot/mobicontrol_data/corpus_mobicontrol.csv")
df = pd.read_csv("/home/ifte/amiebot_project/amie-HelpBot/mobicontrol_data/Train_Test_Data/japanese/corpus_ver14.csv", 
                 header=0, usecols=["page", "text"])

In [ ]:
stop_words_ja = []
STOPWORD_FILE = Path("/home/ifte/amiebot_project/amie-HelpBot/amie_helpbot/" + '/assets/learning/stopword_ja.txt')
with open(STOPWORD_FILE, encoding='utf-8') as fr:
    stop_w = fr.read().splitlines()

In [ ]:
newlist = ['ます', '端末', 'する', '設定', '表示', 'mobicontrol', '選択', '入力', 'アプリ', 'ユーザ', '構成', 'でき', 
           '画面', '接続', 'ルール', 'ない', 'インストール', 'です', '指定', '管理', 'ad', 'ください', 'この', 'クリック', 
           '参照', 'ボタン', 'あり', 'チェック', 'コンソール', 'または', '証明', 'id', '項目', 'なり', 'アクセス',
           'データ', '対象', '適用', 'いる', 'pc', 'その']
stop_words_ja = stop_w + newlist

In [ ]:
import re

def clean_text(text):
    replaced_text = re.sub(r'[【】]', ' ', text)       # 【】の除去
    replaced_text = re.sub(r'[・_]', '', replaced_text)       # ・ の除去
    replaced_text = re.sub(r'[（）()]', ' ', replaced_text)     # （）の除去
    replaced_text = re.sub(r'[［］\[\]]', ' ', replaced_text)   # ［］の除去
    replaced_text = re.sub(r'[@＠]\w+', '', replaced_text)  # メンションの除去
    replaced_text = re.sub(r'https?:\/\/.*?[\r\n ]', '', replaced_text)  # URLの除去
    replaced_text = re.sub(r'　', ' ', replaced_text)  # 全角空白の除去
    replaced_text = re.sub(r'\d+', '', replaced_text) # 数字の除去
    replaced_text = re.sub(r'[-/。,、.=]', ' ', replaced_text)
    
    return replaced_text

In [ ]:
len(stop_words_ja)

In [ ]:
y = df[["page"]]
X = df["text"].apply(lambda x: mecab.parse(x).strip("\n"))
df['parsed'] = X 

In [ ]:
df.parsed

In [ ]:
classes = df.page.unique()

result = pd.DataFrame()

for i in list(classes):
    arr_index = df[df.page == i].parsed.values
    print(arr_index)
    #data = ''.join(list(arr_index))
    break
    #all_data = clean_text(data)
    #print(all_data)
    #break
    
    #arr_words = np.array(all_data.split())
    #print(arr_words) 
    
    #num = arr_words.shape[0]//c_size
    #full = c_size * num
    #rest = arr_words.shape[0] - full
    
    #pad = np.zeros([c_size-rest], dtype=int)
    #sc = np.concatenate((arr_words,pad))
    #features = sc.reshape(num+1, c_size)
    
    df_v = pd.DataFrame(all_data,columns=['data'])
    df_v["cls"] = i
    
    #result = pd.concat([result, df_v]) 
    
    
#print(result)
#result.to_csv('processed.csv')

In [ ]:
k

In [ ]:
k = ''.join(list(arr_index))
df_v = pd.DataFrame(columns=['data'])

In [69]:
df_v.loc[0] = [k]

In [70]:
df_v

,data
0,カスタマイザー から この タイトル を 設定 でき ます 。 カスタマイザー から この ...


In [ ]:
Data_cols = result.iloc[:,:-1]
tog = Data_cols.apply(lambda x: ' '.join(x), axis=1)
#tog.to_csv("tog.csv")
sent = list(tog.values)

In [ ]:
len(sent)
stop_words_ja

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1, 1), stop_words=stop_words_ja, max_df=0.95)
vectors = vectorizer.fit_transform(sent)

In [ ]:
vectorizer

In [ ]:
# idf score 
#idf = vectorizer.idf_
# visulalize idf score
#print(dict(zip(vectorizer.get_feature_names(), idf)))

weights = np.asarray(vectors.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': vectorizer.get_feature_names(), 'weight': weights})

In [ ]:
TermsWeight = weights_df.sort_values(by='weight', ascending=False).head(50)

In [ ]:
TermsWeight.term

In [ ]:
# Read a question from the user
question = [input('Please enter a question: \n')]
question = vectorizer.transform(question)

# Rank all the questions using cosine similarity to the input question
rank = cosine_similarity(question, vectorizer.transform(sent))

# Grab the top 5
top = np.argsort(rank, axis=-1).T[-10:]
#print(top)

for item in range(len(top)):
    idx = top[item,0]
    
    # if the dataset file structure "Quesition | Answer" then use following line  
    #print(data['Answer'].iloc[item].values[0])
    print("PageID: ",result.iloc[idx,-1],"idx",idx,"=",tog.values[idx])
    print("\n ########## \n")